In [49]:
import pandas as pd
import numpy as np
import re
import nltk
import string 
import preprocessor as p 
from nltk.corpus import stopwords
from nltk.stem.porter import *
from nltk.tokenize import word_tokenize

In [55]:
input_data = []

with open('./sts-2017-en-es/En_Es_STS/STS.input.en-es.train.txt', 'r') as inFile:
    input_data = inFile.readlines()

X = []
for data in input_data:
    X.append(data.split('\t')[:2])

In [58]:
X = [[x[0],x[1]] for x in X]

cleaned_eng = []
cleaned_esp = []
for x in X:
    eng, esp = x[0], x[1]
    eng, esp = p.clean(eng).lower(), p.clean(esp).lower() #cleaning
    for c in string.punctuation: #removing punctuations
        if c in eng:
            eng = eng.replace(c,'')
        if c in esp:
            esp = esp.replace(c,'')
    
    eng, esp = word_tokenize(eng), word_tokenize(esp) #tokenizing
    cleaned_eng.append(eng)
    cleaned_esp.append(esp)


In [59]:
# make vocab
eng_vocab = {}
index=0
for sentence in cleaned_eng:
    for word in sentence:
        if word in eng_vocab:
            continue
        else:
            eng_vocab[word] = index
            index+=1

esp_vocab = {}
index=0
for sentence in cleaned_eng:
    for word in sentence:
        if word in esp_vocab:
            continue
        else:
            esp_vocab[word] = index
            index+=1


['his', 'rheumy', 'eyes', 'began', 'to', 'cloud']

In [ ]:
pr